In [98]:
import torch

## Creating Layers

In [109]:
class DenseLayer:
  # Layer initialization
  def __init__(self, n_inputs, n_neurons):
    # Initialize weights and biases
    self.weights = 0.01 * torch.rand(n_inputs, n_neurons)
    self.biases = torch.zeros((1, n_neurons))

  # Forward pass
  def forward(self, inputs):
    # Calculate output values from inputs, weights and biases
    self.output = torch.matmul(inputs, self.weights) + self.biases

## Activation Functions

In [110]:
class Activation_ReLU:
  # Forward pass
  def forward(self, inputs):
    self.output = torch.max(torch.tensor(0),inputs)
class Activation_Sigmoid:
  # Forward pass
  def forward(self, inputs):
    self.output = 1 / (1 + torch.exp(inputs*-1))
class Activation_Softmax:
  # Forward pass
  def forward(self, inputs):
    # Get unnormalized probabilities
    exp_values = torch.exp(inputs - torch.max(inputs, axis=1, keepdim=True).values)
    # Normalize them for each sample
    probabilities = exp_values / torch.sum(exp_values, axis=1, keepdim=True)
    self.output = probabilities
class Activation_Linear:
    def forward(self, inputs):
        self.output = inputs

## Loss and accuracy

In [111]:
class Accuracy():
  def calculate(self, y_pred, y_true):
    predictions = torch.argmax(y_pred, axis=1)
    if len(y_true.shape) == 2:
      y_true = torch.argmax(y_true, axis=1)
    accuracy = torch.mean((predictions == y_true).float())
    return accuracy
def MSE(y_true, y_pred):
  return torch.mean(0.5*(y_true - y_pred)**2)

In [112]:
X = torch.tensor([0.1, 0.5])
y = torch.tensor([0.05,0.2, 0.45, 0.3])

In [113]:
hidden_layer_1 = DenseLayer(2, 4)
activation1 = Activation_Sigmoid()
output_layer = DenseLayer(4, 4)
activation2 = Activation_Linear()

In [114]:
def forward_pass(X):
  hidden_layer_1.forward(X)
  activation1.forward(hidden_layer_1.output)
  output_layer.forward(activation1.output)
  activation2.forward(output_layer.output)
  return activation2.output

In [115]:
def back_prop(fp):
    lr = torch.tensor(0.01)
    gradients_output = fp - y
    output_layer.weights -= lr * torch.matmul(activation1.output.T, gradients_output)
    output_layer.biases -= lr * gradients_output.sum(dim=0, keepdim=True)
    gradients_hidden = torch.matmul(gradients_output, output_layer.weights.T) * activation1.output * (1 - activation1.output)
    hidden_layer_1.weights -= lr * torch.matmul(X.reshape(-1, 1), gradients_hidden)
    hidden_layer_1.biases -= lr * gradients_hidden.sum(dim=0, keepdim=True)


In [116]:
loss = 0.0001
y_pred = forward_pass(X)
err = MSE(y, y_pred)
print("Initial loss:", err)
print("Initial prediction:",y_pred)
epoch = 1000
for i in range(epoch):
  back_prop(y_pred)
  y_pred = forward_pass(X)
  err = MSE(y, y_pred)
  epoch += 1
  if i%100 == 0:
    # let me check the loss the prediction in every 100 iterations
    print("-----------------------------------------")
    print("Current loss:", err)
    print("Current prediction:",y_pred)
    print("Target value:",y)

Initial loss: tensor(0.0399)
Initial prediction: tensor([[0.0103, 0.0057, 0.0096, 0.0071]])
-----------------------------------------
Current loss: tensor(0.0383)
Current prediction: tensor([[0.0111, 0.0096, 0.0184, 0.0130]])
Target value: tensor([0.0500, 0.2000, 0.4500, 0.3000])
-----------------------------------------
Current loss: tensor(0.0007)
Current prediction: tensor([[0.0449, 0.1749, 0.3932, 0.2622]])
Target value: tensor([0.0500, 0.2000, 0.4500, 0.3000])
-----------------------------------------
Current loss: tensor(1.1344e-05)
Current prediction: tensor([[0.0493, 0.1967, 0.4426, 0.2951]])
Target value: tensor([0.0500, 0.2000, 0.4500, 0.3000])
-----------------------------------------
Current loss: tensor(1.9338e-07)
Current prediction: tensor([[0.0499, 0.1996, 0.4490, 0.2994]])
Target value: tensor([0.0500, 0.2000, 0.4500, 0.3000])
-----------------------------------------
Current loss: tensor(3.2958e-09)
Current prediction: tensor([[0.0500, 0.1999, 0.4499, 0.2999]])
Target

In [117]:
acc = Accuracy()
accuracy = acc.calculate(y_pred, y.unsqueeze(0))
print("The accuracy is:", accuracy)

The accuracy is: tensor(1.)
